<a href="https://colab.research.google.com/github/Mariacfagundes/Desafio-Alura/blob/main/Desafio_O_Impacto_do_Envelhecimento_Populacional_no_Brasil_(Censo_2022).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tratar a Tabela 10291 (renda da população 60+)**

In [7]:
df_domicilios = pd.read_csv("tabela10175.csv", sep=";", encoding="utf-8-sig", skiprows=5)
df_envelhecimento = pd.read_csv("tabela9515.csv", sep=";", encoding="utf-8-sig", skiprows=5)
df_renda_idosos = pd.read_csv("tabela10291.csv", sep=";", encoding="utf-8-sig", skiprows=5)

In [9]:
df_renda_idosos.columns = [
    "Nível", "Código", "Município", "Faixa etária", "Renda média"
]

In [10]:
df_renda_idosos = df_renda_idosos[df_renda_idosos["Nível"] == "MU"]

In [12]:
# Substituir traços por NaN e converter para float
df_renda_idosos["Renda média"] = (
    df_renda_idosos["Renda média"]
    .replace("-", None)
    .str.replace(",", ".", regex=False)
    .astype(float)
)

In [13]:
df_renda_idosos = (
    df_renda_idosos.groupby("Município")["Renda média"]
    .mean()
    .reset_index()
    .rename(columns={"Renda média": "Renda média 60+"})
)

In [27]:
df_renda_idosos.head()



,Município,Renda média 60+
0,Abadia de Goiás (GO),2198.215
1,Abadia dos Dourados (MG),2849.805
2,Abadiânia (GO),2485.185
3,Abaetetuba (PA),1414.680
4,Abaeté (MG),2449.180


# **Tratar a Tabela 9515 — Índice de Envelhecimento**

In [15]:
df_envelhecimento.columns = [
    "Nível", "Código", "Município", "Índice de envelhecimento"
]

In [16]:
df_envelhecimento = df_envelhecimento[df_envelhecimento["Nível"] == "MU"]

In [17]:
df_envelhecimento["Índice de envelhecimento"] = (
    df_envelhecimento["Índice de envelhecimento"]
    .str.replace(",", ".", regex=False)
    .astype(float)
)

In [18]:
df_envelhecimento.head()

,Nível,Código,Município,Índice de envelhecimento
0,MU,1100023,Ariquemes (RO),34.61
1,MU,1100031,Cabixi (RO),64.14
2,MU,1100049,Cacoal (RO),43.50
3,MU,1100056,Cerejeiras (RO),47.01
4,MU,1100064,Colorado do Oeste (RO),60.62


# **Tabela 10175 — Estimativa de casais sem filhos**


In [19]:
df_domicilios.columns = [
    "Nível", "Código", "Município", "Número de componentes", "Total", "Única"
]

In [20]:
df_domicilios = df_domicilios[df_domicilios["Nível"] == "MU"]

In [21]:
df_2pessoas = df_domicilios[df_domicilios["Número de componentes"] == "2 pessoas"]

In [23]:
# Substituir traços por NaN e converter para float
df_2pessoas["Total"] = (
    df_2pessoas["Total"]
    .replace("-", None)
    .str.replace(",", ".", regex=False)
    .astype(float)
)

df_2pessoas["Única"] = (
    df_2pessoas["Única"]
    .replace("-", None)
    .str.replace(",", ".", regex=False)
    .astype(float)
)

/tmp/ipython-input-603981308.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2pessoas["Total"] = (
/tmp/ipython-input-603981308.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2pessoas["Única"] = (


In [24]:
df_2pessoas["Proporção casais sem filhos"] = df_2pessoas["Única"] / df_2pessoas["Total"]

/tmp/ipython-input-2296601309.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2pessoas["Proporção casais sem filhos"] = df_2pessoas["Única"] / df_2pessoas["Total"]


In [25]:
df_casais = df_2pessoas[[
    "Município", "Total", "Única", "Proporção casais sem filhos"
]].copy()

In [26]:
df_casais.head()

,Município,Total,Única,Proporção casais sem filhos
2,Alta Floresta D'Oeste (RO),2671.0,2430.0,0.909772
4,Ariquemes (RO),11780.0,10102.0,0.857555
6,Cabixi (RO),697.0,646.0,0.926829
8,Cacoal (RO),11166.0,9784.0,0.876231
10,Cerejeiras (RO),2022.0,1882.0,0.930762


# **Unir as três tabelas**


In [28]:
# Unir casais sem filhos + índice de envelhecimento
df_final = df_casais.merge(df_envelhecimento, on="Município", how="inner")

# Unir com a renda média 60+
df_final = df_final.merge(df_renda_idosos, on="Município", how="inner")

In [29]:
df_final.head()

,Município,Total,Única,Proporção casais sem filhos,Nível,Código,Índice de envelhecimento,Renda média 60+
0,Alta Floresta D'Oeste (RO),2671.0,2430.0,0.909772,MU,1100015,34.00,2407.27
1,Alta Floresta D'Oeste (RO),2671.0,2430.0,0.909772,MU,1100015,101.82,2407.27
2,Ariquemes (RO),11780.0,10102.0,0.857555,MU,1100023,34.61,2420.52
3,Ariquemes (RO),11780.0,10102.0,0.857555,MU,1100023,32.00,2420.52
4,Ariquemes (RO),11780.0,10102.0,0.857555,MU,1100023,98.25,2420.52


# **Construção do Dashboard com Streamlit**
